##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/building_your_own_federated_learning_algorithm"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.36.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.36.0/docs/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>   </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 시작하기 전에

시작하기 전에 다음을 실행하여 환경이 올바르게 설정되었는지 확인하세요. 인사말이 표시되지 않으면 [설치](../install.md) 가이드에서 지침을 참조하세요. 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

**참고**: 이 Colab은 <code>tensorflow_federated</code> pip 패키지의 <a>최신 릴리즈 버전</a>에서 동작하는 것으로 확인되었지만 Tensorflow Federated 프로젝트는 아직 시험판 개발 중이며 `main`에서 동작하지 않을 수 있습니다.

# 고유한 페더레이션 학습 알고리즘 구축하기

In the [image classification](federated_learning_for_image_classification.ipynb) and [text generation](federated_learning_for_text_generation.ipynb) tutorials, you learned how to set up model and data pipelines for Federated Learning (FL), and performed federated training via the `tff.learning` API layer of TFF.

This is only the tip of the iceberg when it comes to FL research. This tutorial discusses how to implement federated learning algorithms *without* deferring to the `tff.learning` API. In this tutorial, you will accomplish the following:

**목표**

- 페더레이션 학습 알고리즘의 일반적인 구조를 이해합니다.
- TFF의 *페더레이션 코어*를 탐색합니다.
- Federated Core를 사용하여 Federated Averaging을 직접 구현합니다.

While this tutorial is self-contained, it may be useful to first check out the [image classification](federated_learning_for_image_classification.ipynb) and [text generation](federated_learning_for_text_generation.ipynb) tutorials.


## 입력 데이터 준비하기

First load and preprocess the EMNIST dataset included in TFF. For more details, see the [image classification](federated_learning_for_image_classification.ipynb) tutorial.

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

In order to feed the dataset into our model, the data is flattened, and each example is converted into a tuple of the form `(flattened_image_vector, label)`.

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

Now, select a small number of clients, and apply the preprocessing above to their datasets.

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## 모델 준비하기

This uses the same model as in the [image classification](federated_learning_for_image_classification.ipynb) tutorial. This model (implemented via `tf.keras`) has a single hidden layer, followed by a softmax layer.

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

In order to use this model in TFF, wrap the Keras model as a [`tff.learning.Model`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model). This allows one to perform the model's [forward pass](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass) within TFF, and [extract model outputs](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics). For more details, also see the [image classification](federated_learning_for_image_classification.ipynb) tutorial.

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

While the above used `tf.keras` to create a `tff.learning.Model`, TFF supports much more general models. These models have the following relevant attributes capturing the model weights:

- `trainable_variables`: 학습 가능한 레이어에 해당하는 텐서의 이터러블입니다.
- `non_trainable_variables`: 학습할 수 없는 레이어에 해당하는 텐서의 이터러블입니다.

In this tutorial, only the `trainable_variables` will be used. (as the model only has those!).

# 고유한 페더레이션 학습 알고리즘 구축하기

`tff.learning` API를 사용하면 Federated Averaging의 많은 변형을 생성할 수 있지만 이 프레임워크에 깔끔하게 맞지 않는 다른 페더레이션 알고리즘이 있습니다. 예를 들어 정규화, 클리핑 또는 [페더레이션 GAN 학습](https://github.com/tensorflow/federated/tree/main/tensorflow_federated/python/research/gans)과 같은 더 복잡한 알고리즘을 추가할 수도 있습니다. 아니면 [페더레이션 분석](https://ai.googleblog.com/2020/05/federated-analytics-collaborative-data.html)에 관심이 있을 수도 있습니다.

For these more advanced algorithms, you'll have to write our own custom algorithm using TFF. In many cases, federated algorithms have 4 main components:

1. 서버-클라이언트 브로드캐스트 단계
2. 로컬 클라이언트 업데이트 단계
3. 클라이언트-서버 업로드 단계
4. 서버 업데이트 단계

In TFF, a federated algorithm is typically represented as a [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess) (which will be referred to as just an `IterativeProcess` throughout). This is a class that contains `initialize` and `next` functions. Here, `initialize` is used to initialize the server, and `next` will perform one communication round of the federated algorithm. Let's write a skeleton of what our iterative process for FedAvg should look like.

First, there is an initialize function that simply creates a `tff.learning.Model`, and returns its trainable weights.

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

This function looks good, but as you will see later, you will need to make a small modification to make it a "TFF computation".

Next, let's write a sketch of the `next_fn`.

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

Let's focus on implementing these four components separately. First, let's focus on the parts that can be implemented in pure TensorFlow, namely the client and server update steps.


## TensorFlow 블록 

### 클라이언트 업데이트

The `tff.learning.Model` can be used to do client training in essentially the same way you would train a TensorFlow model. In particular, one can use `tf.GradientTape` to compute the gradient on batches of data, then apply these gradient using a `client_optimizer`. This will only involve the trainable weights.


In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### 서버 업데이트

The server update for FedAvg is simpler than the client update. This tutorial will implement "vanilla" federated averaging, in which the server model weights are replaced by the average of the client model weights. Again, this only uses the trainable weights.

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

단순히 `mean_client_weights`를 반환하여 스니펫을 단순화할 수 있습니다. 그러나 Federated Averaging의 고급 구현에서는 모멘텀 또는 적응성과 같은 보다 정교한 기술과 함께 `mean_client_weights`를 사용합니다.

**과제**: model_weights 및 mean_client_weights의 중간점이 되도록 서버 가중치를 업데이트하는 `server_update` 버전을 구현합니다(참고: 이러한 종류의 "중간점" 접근 방식은 [Lookahead 옵티마이저](https://arxiv.org/abs/1907.08610)에 대한 최근의 작업과 유사합니다!).

So far, this has only involved TensorFlow code. This is by design, as TFF allows you to use much of the TensorFlow code you're already familiar with. Next you will have to specify the **orchestration logic**, that is, the logic that dictates what the server broadcasts to the client, and what the client uploads to the server.

이를 위해서는 TFF의 *페더레이션 코어*가 필요합니다.

# 페더레이션 코어 소개

페더레이션 코어(FC)는 `tff.learning` API의 기반 역할을 하는 하위 수준 인터페이스 집합입니다. 그러나 이러한 인터페이스는 학습에만 국한되지 않습니다. 실제로 분산 데이터에 대한 분석 및 기타 많은 계산에 사용할 수 있습니다.

상위 수준에서 페더레이션 코어는 간결하게 표현된 프로그램 로직을 사용하여 TensorFlow 코드를 분산 통신 연산자(예: 분산 합계 및 브로드캐스트)와 결합할 수 있는 개발 환경입니다. 목표는 시스템 구현 세부 사항(예: 지점 간 네트워크 메시지 교환 지정)을 요구하지 않고 연구자와 실무자에게 시스템의 분산 통신을 신속하게 제어할 수 있도록 하는 것입니다.

한 가지 요점은 TFF가 개인 정보 보호를 위해 설계되었다는 것입니다. 따라서 중앙 집중식 서버 위치에서 원치 않는 데이터 축적을 방지하기 위해 데이터가 있는 위치를 명시적으로 제어할 수 있습니다.

## 페더레이션 데이터

A key concept in TFF is "federated data", which refers to a collection of data items hosted across a group of devices in a distributed system (eg. client datasets, or the server model weights). The entire collection of values across all devices is represented as a single *federated value*.

For example, suppose there are client devices that each have a float representing the temperature of a sensor. These floats can be represented as a *federated float* by

In [ ]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

Federated types are specified by a type `T` of its member constituents (eg. `tf.float32`) and a group `G` of devices. Typically, `G` is either `tff.CLIENTS` or `tff.SERVER`. Such a federated type is represented as `{T}@G`, as shown below.

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

Why does TFF care so much about placements? A key goal of TFF is to enable writing code that could be deployed on a real distributed system. This means that it is vital to reason about which subsets of devices execute which code, and where different pieces of data reside.

TFF는 *데이터*, 데이터가 *배치되는 위치* 및 데이터가 *변환*되는 방법의 세 가지에 중점을 둡니다. 처음 두 개는 페더레이션 유형으로 캡슐화되고 마지막 두 개는 *페더레이션 계산*에 캡슐화됩니다.

## 페더레이션 계산

TFF는 기본 단위가 *페더레이션 계산*인 강력한 형식의 함수형 프로그래밍 환경입니다. 이들은 페더레이션 값을 입력으로 받아들이고 페더레이션 값을 출력으로 반환하는 논리 조각입니다.

For example, suppose you wanted to average the temperatures on our client sensors. You could define the following (using our federated float):

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

TensorFlow의 `tf.function` 데코레이터와 어떻게 다른지 물어볼 수 있습니다. 핵심적인 대답은 `tff.federated_computation`에 의해 생성된 코드가 TensorFlow도 아니고 Python 코드도 아니라는 것입니다. 내부 플랫폼 독립적인 *글루 언어*로 된 분산 시스템의 사양입니다.

복잡하게 들릴 수 있지만 TFF 계산은 잘 정의된 형식 서명이 있는 함수로 생각할 수 있습니다. 이러한 유형 서명은 직접 쿼리할 수 있습니다.

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

이 `tff.federated_computation`은 페더레이션 유형 `<float>@CLIENTS`의 인수를 받아들이고 페더레이션 유형 `<float>@SERVER`의 값을 반환합니다. 페더레이션 계산은 서버에서 클라이언트로, 클라이언트에서 클라이언트로 또는 서버에서 서버로 이동할 수도 있습니다. 페더레이션 계산은 유형 서명이 일치하는 한 일반 함수처럼 구성할 수도 있습니다.

To support development, TFF allows you to invoke a `tff.federated_computation` as a Python function. For example, you can call

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

## 비 즉시 실행 계산 및 TensorFlow

두 가지 주요 제한 사항을 알고 있어야 합니다. 첫째, Python 인터프리터가 `tff.federated_computation` 데코레이터를 만나면 함수가 한 번 추적되고 나중에 사용할 수 있도록 직렬화됩니다. 페더레이션 학습의 분산된 특성으로 인해 이러한 미래의 사용은 원격 실행 환경과 같은 다른 곳에서 발생할 수 있습니다. 따라서 TFF 계산은 기본적으로 *비 즉시 실행*입니다. 이 동작은 TensorFlow의 [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) 데코레이터와 다소 유사합니다.

둘째, 페더레이션 계산은 페더레이션 연산자(예: `tff.federated_mean`)로만 구성될 수 있으며 TensorFlow 작업을 포함할 수 없습니다. TensorFlow 코드는 `tff.tf_computation` 데코레이션된 블록으로 제한되어야 합니다. 숫자를 취하고 여기에 `0.5`를 더하는 다음 함수와 같이 대부분의 일반적인 TensorFlow 코드는 직접 데코레이션할 수 있습니다.

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

These also have type signatures, but *without placements*. For example, you can call

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

This showcases an important difference between `tff.federated_computation` and `tff.tf_computation`. The former has explicit placements, while the latter does not.

You can use `tff.tf_computation` blocks in federated computations by specifying placements. Let's create a function that adds half, but only to federated floats at the clients. You can do this by using `tff.federated_map`, which applies a given `tff.tf_computation`, while preserving the placement.

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

This function is almost identical to `add_half`, except that it only accepts values with placement at `tff.CLIENTS`, and returns values with the same placement. This can be seen in its type signature:

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

요약하자면:

- TFF는 페더레이션 값에서 작동합니다.
- 각 페더레이션 값에는 *유형*(예: `tf.float32`)과 *배치*(예: `tff.CLIENTS`)가 있는 *페더레이션 유형{/em}이 있습니다.*
- 페더레이션 값은 *페더레이션 계산*을 사용하여 변환될 수 있으며, `tff.federated_computation` 및 연합 유형 서명으로 데코레이션되어야 합니다.
- TensorFlow 코드는 `tff.tf_computation` 데코레이터가 있는 블록에 포함되어야 합니다.
- 그런 다음 이러한 블록을 페더레이션 계산에 통합할 수 있습니다.


# 고유한 페더레이션 학습 알고리즘 구축하기, 재검토

Now that you've gotten a glimpse of the Federated Core, you can build our own federated learning algorithm. Remember that above, you defined an `initialize_fn` and `next_fn` for our algorithm. The `next_fn` will make use of the `client_update` and `server_update` you defined using pure TensorFlow code.

However, in order to make our algorithm a federated computation, you will need both the `next_fn` and `initialize_fn` to each be a `tff.federated_computation`.

## TensorFlow 페더레이션 블록 

### 초기화 계산 만들기

The initialize function will be quite simple: You will create a model using `model_fn`. However, remember that you must separate out our TensorFlow code using `tff.tf_computation`.

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

You can then pass this directly into a federated computation using `tff.federated_value`.

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

### `next_fn` 만들기

The client and server update code can now be used to write the actual algorithm. First, you will turn the `client_update` into a `tff.tf_computation` that accepts a client datasets and server weights, and outputs an updated client weights tensor.

You will need the corresponding types to properly decorate our function. Luckily, the type of the server weights can be extracted directly from our model.

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

Let's look at the dataset type signature. Remember that you took 28 by 28 images (with integer labels) and flattened them.

In [ ]:
str(tf_dataset_type)

'<float32[?,784],int32[?,1]>*'

You can also extract the model weights type by using our `server_init` function above.

In [ ]:
model_weights_type = server_init.type_signature.result

Examining the type signature, you'll be able to see the architecture of our model!

In [ ]:
str(model_weights_type)

'<float32[784,10],float32[10]>'

You can now create our `tff.tf_computation` for the client update.

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

The `tff.tf_computation` version of the server update can be defined in a similar way, using types you've already extracted.

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

Last, but not least, you need to create the `tff.federated_computation` that brings this all together. This function will accept two *federated values*, one corresponding to the server weights (with placement `tff.SERVER`), and the other corresponding to the client datasets (with placement `tff.CLIENTS`).

Note that both these types were defined above! You simply need to give them the proper placement using `tff.FederatedType`.

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

FL 알고리즘의 4가지 요소를 기억하십니까?

1. 서버-클라이언트 브로드캐스트 단계
2. 로컬 클라이언트 업데이트 단계
3. 클라이언트-서버 업로드 단계
4. 서버 업데이트 단계

Now that you've built up the above, each part can be compactly represented as a single line of TFF code. This simplicity is why you had to take extra care to specify things such as federated types!

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

You now have a `tff.federated_computation` for both the algorithm initialization, and for running one step of the algorithm. To finish our algorithm, you pass these into `tff.templates.IterativeProcess`.

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

반복 프로세스의 <code>initialize</code> 및 `next` 함수의 <em>유형 서명</em>을 살펴보겠습니다.

In [ ]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[784,10],float32[10]>@SERVER)'

이는 `federated_algorithm.initialize`가 단일 레이어 모델(784x10 가중치 행렬 및 10개의 바이어스 단위 포함)을 반환하는 인수가 없는 함수라는 사실을 반영합니다.

In [ ]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[784,10],float32[10]>@SERVER,federated_dataset={<float32[?,784],int32[?,1]>*}@CLIENTS> -> <float32[784,10],float32[10]>@SERVER)'

Here, one can see that `federated_algorithm.next` accepts a server model and client data, and returns an updated server model.

## 알고리즘 평가

Let's run a few rounds, and see how the loss changes. First, you will define an evaluation function using the *centralized* approach discussed in the second tutorial.

You will first create a centralized evaluation dataset, and then apply the same preprocessing you used for the training data.

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

Next, you will write a function that accepts a server state, and uses Keras to evaluate on the test dataset. If you're familiar with `tf.Keras`, this will all look familiar, though note the use of `set_weights`!

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

이제 알고리즘을 초기화하고 테스트세트에서 평가해 보겠습니다.

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

2042/2042 [==============================] - 2s 767us/step - loss: 2.8479 - sparse_categorical_accuracy: 0.1027


몇 라운드 동안 훈련하고 변경 사항이 있는지 살펴보겠습니다.

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 2s 738us/step - loss: 2.5867 - sparse_categorical_accuracy: 0.0980


There is a slight decrease in the loss function. While the jump is small, you've only performed 15 training rounds, and on a small subset of clients. To see better results, you may have to do hundreds if not thousands of rounds.

## 알고리즘 수정

At this point, let's stop and think about what you've accomplished. You've implemented Federated Averaging directly by combining pure TensorFlow code (for the client and server updates) with federated computations from the Federated Core of TFF.

To perform more sophisticted learning, you can simply alter what you have above. In particular, by editing the pure TF code above, you can change how the client performs training, or how the server updates its model.

**도전 과제:** <code>client_update</code> 함수에 <a>그레디언트 클리핑</a>을 추가합니다.


If you wanted to make larger changes, you could also have the server store and broadcast more data. For example, the server could also store the client learning rate, and make it decay over time! Note that this will require changes to the type signatures used in the `tff.tf_computation` calls above.

**더 어려운 과제:** 클라이언트에서 학습률 감소를 사용하여 Federated Averaging을 구현합니다.

At this point, you may begin to realize how much flexibility there is in what you can implement in this framework. For ideas (including the answer to the harder challenge above) you can see the source-code for [`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg), or check out various [research projects](https://github.com/google-research/federated) using TFF.